# Demo Notebook for CS411 Project Track 1

Since we do not have an interface for this project because of our focus on the algorithm itself, we put the basic functionalities needed for Project Track 1 here as well as any additional progress we have made for our project.

In [7]:
# Import statements
import crud_operations as co # this imports our crud operations class

## CRUD Operations

This next section shows off our basic CRUD operations. We used a script called `csv_to_db.py` that inputted all of our data (saved as a CSV) to the MySQL database. Below, we show off some of the different functionality that we have. To see the queries we are using, you can look at `crud_operations.py`.

For this database, we have two tables:
* Queries(queryId, questionId, queryContent)
* Questions(questionId, questionContent)

### Create/Read

In [8]:
# First, create an instance we can use that uses SampleQueries db
db = co.MySQLDB("SampleQueries")

In [9]:
# Read current database state
print("This is reading the Questions in our DB:")
for entry in db.read_all_questions():
    print(entry)

This is reading the Questions in our DB:
(1, 'Popular Name')
(2, 'Best Performer')
(3, 'Course Enrollment')
(4, 'test question insert')
(5, 'test question insert')
(6, 'test question insert')
(7, 'test question insert')
(8, 'Quarantine make you hungry?')
(9, 'Is pizza breakfast?')
(10, 'test question insert')
(11, 'test question insert')
(12, 'test question insert')
(14, 'What is our professors name?')
(15, 'What is our professors name?')


In [10]:
# Read current database queries
print("This is reading the Queries in our DB:")
all_queries = db.read_all_queries()
for entry in all_queries:
    print(entry)

This is reading the Queries in our DB:
(1, 1, 'SELECT Firstname FROM Students WHERE Department="Statistics" AND LastName LIKE \'A%\' ORDER BY FirstName DESC')
(2, 1, '\nSELECT Firstname\nFROM Students\nWHERE Department="Statistics" AND LastName LIKE \'A%\'\nORDER BY FirstName\n')
(3, 2, " Select NetId from Students NATURAL JOIN Enrollments where (CRN, Score) in (Select E.CRN, max(Score) From Enrollments E, Courses C Where E.CRN = C.CRN and C.Department = 'CS' Group By E.CRN) AND Students.Department <> 'CS'")
(4, 2, " Select NetID from Enrollments where [CRN, Score] in (Select E.CRN, max(Score) From Enrollments E, Courses C Where E.CRN = C.CRN and C.Department = 'CS' Group By E.CRN))")
(5, 2, "Select temp.CRN, max(temp.Score) from (Select CRN, NetId, Score From Enrollments E NATURAL JOIN Courses C Where C.Department = 'CS') as temp, Students as S Where S.Department <> 'CS'and S.NetId=temp.NetId GROUP BY temp.CRN")
(6, 2, "Select NetId from (Select NetID, CRN, Score from Enrollments wher

In [11]:
print("Number of queries:", len(all_queries))

Number of queries: 116


We can also insert into the database by creating another question or query.

In [12]:
print("This is inserting a new Question")
question = input("Enter a question: ")
db.create_question(question)

This is inserting a new Question
Enter a question: What is today's date?
Inserted the question: What is today's date?


In [13]:
# We can check by using our read functionality
for entry in db.read_all_questions():
    print(entry)

(1, 'Popular Name')
(2, 'Best Performer')
(3, 'Course Enrollment')
(4, 'test question insert')
(5, 'test question insert')
(6, 'test question insert')
(7, 'test question insert')
(8, 'Quarantine make you hungry?')
(9, 'Is pizza breakfast?')
(10, 'test question insert')
(11, 'test question insert')
(12, 'test question insert')
(14, 'What is our professors name?')
(15, 'What is our professors name?')
(16, "What is today's date?")


In [14]:
# If we do the same for queries, we can see we get an additional query in the DB
db.create_query(1, "SELECT teacherName FROM teachers WHERE teacherName = 'Abdu'") # adds query for question 1

Inserted "SELECT teacherName FROM teachers WHERE teacherName = 'Abdu'" for question 1


In [15]:
new_all_queries = db.read_all_queries()
print("Number of queries:", len(new_all_queries)) # Should be 1 more than last time we looked

Number of queries: 117


### Update

In [16]:
print(db.read_all_questions())
qid = input("Enter question ID to be updated: ")
q = input("Enter new value for question: ")
db.update_question_by_id(qid, q)

((1, 'Popular Name'), (2, 'Best Performer'), (3, 'Course Enrollment'), (4, 'test question insert'), (5, 'test question insert'), (6, 'test question insert'), (7, 'test question insert'), (8, 'Quarantine make you hungry?'), (9, 'Is pizza breakfast?'), (10, 'test question insert'), (11, 'test question insert'), (12, 'test question insert'), (14, 'What is our professors name?'), (15, 'What is our professors name?'), (16, "What is today's date?"))
Enter question ID to be updated: 1
Enter new value for question: Popular Name
Updated Questions for questionID=  1


In [17]:
print(db.read_all_questions())

((1, 'Popular Name'), (2, 'Best Performer'), (3, 'Course Enrollment'), (4, 'test question insert'), (5, 'test question insert'), (6, 'test question insert'), (7, 'test question insert'), (8, 'Quarantine make you hungry?'), (9, 'Is pizza breakfast?'), (10, 'test question insert'), (11, 'test question insert'), (12, 'test question insert'), (14, 'What is our professors name?'), (15, 'What is our professors name?'), (16, "What is today's date?"))


### Delete

In [18]:
qid = input("Enter question ID to be deleted (Questions Table): ")
db.delete_question_by_id(qid)

Enter question ID to be deleted (Questions Table): 16
Deleted from Questions for questionID=  16


In [19]:
print("After Delete")
print(db.read_all_questions())

After Delete
((1, 'Popular Name'), (2, 'Best Performer'), (3, 'Course Enrollment'), (4, 'test question insert'), (5, 'test question insert'), (6, 'test question insert'), (7, 'test question insert'), (8, 'Quarantine make you hungry?'), (9, 'Is pizza breakfast?'), (10, 'test question insert'), (11, 'test question insert'), (12, 'test question insert'), (14, 'What is our professors name?'), (15, 'What is our professors name?'))


## Analysis on SQL Queries 

In [31]:
import csv
import nltk

In [32]:
## Get keywords 
keywords=set()
with open('sql_reserved_words.csv') as words:
    reader = csv.reader(words)
    for row in reader:
        keywords.add(row[0])

In [33]:
# Read current database queries
raw = db.read_all_queries()
data= []
for entry in raw:
    data.append([entry[0], entry[2].strip()])

In [35]:
# Contents of data:
# 0: qId
# 1: Original query
# 2: Ordered array of keyword occurences
# 3: Dictionary of filtered keywords, by count


for d in data:
    t = nltk.word_tokenize(d[1])
    filtered=[]
    dic={}
    for w in t: 
        word=w.upper()
        if word in keywords:
            filtered.append(word)
            if word in dic:
                dic[word]+=1
            else:
                dic[word]=1
    d.append(filtered)
    d.append(dic)

In [40]:
# Preview ordered list of keywords 
print("Ordered array of keywords occurences for each query: ")
for idx, d in enumerate(data):
    print(d[2])
    
    if idx > 5: 
        break

Ordered array of keywords occurences for each query: 
['SELECT', 'FROM', 'WHERE', 'STATISTICS', 'AND', 'LIKE', 'A', 'ORDER', 'BY', 'DESC']
['SELECT', 'FROM', 'WHERE', 'STATISTICS', 'AND', 'LIKE', 'A', 'ORDER', 'BY']
['SELECT', 'FROM', 'NATURAL', 'JOIN', 'WHERE', 'IN', 'SELECT', 'MAX', 'FROM', 'C', 'WHERE', 'AND', 'GROUP', 'BY', 'AND']
['SELECT', 'FROM', 'WHERE', 'IN', 'SELECT', 'MAX', 'FROM', 'C', 'WHERE', 'AND', 'GROUP', 'BY']
['SELECT', 'MAX', 'FROM', 'SELECT', 'FROM', 'NATURAL', 'JOIN', 'C', 'WHERE', 'AS', 'TEMP', 'AS', 'WHERE', 'GROUP', 'BY']
['SELECT', 'FROM', 'SELECT', 'FROM', 'WHERE', 'IN', 'SELECT', 'FROM', 'WHERE', 'AS', 'TEMP', 'WHERE', 'MAX']
['SELECT', 'FROM', 'NATURAL', 'JOIN', 'C', 'WHERE']


In [42]:
# Print dictionary counts of keywords
print("Counts of each keyword for each query: ")
for idx, d in enumerate(data):
    print(d[3])
    
    if idx > 5: 
        break

Counts of each keyword for each query: 
{'SELECT': 1, 'FROM': 1, 'WHERE': 1, 'STATISTICS': 1, 'AND': 1, 'LIKE': 1, 'A': 1, 'ORDER': 1, 'BY': 1, 'DESC': 1}
{'SELECT': 1, 'FROM': 1, 'WHERE': 1, 'STATISTICS': 1, 'AND': 1, 'LIKE': 1, 'A': 1, 'ORDER': 1, 'BY': 1}
{'SELECT': 2, 'FROM': 2, 'NATURAL': 1, 'JOIN': 1, 'WHERE': 2, 'IN': 1, 'MAX': 1, 'C': 1, 'AND': 2, 'GROUP': 1, 'BY': 1}
{'SELECT': 2, 'FROM': 2, 'WHERE': 2, 'IN': 1, 'MAX': 1, 'C': 1, 'AND': 1, 'GROUP': 1, 'BY': 1}
{'SELECT': 2, 'MAX': 1, 'FROM': 2, 'NATURAL': 1, 'JOIN': 1, 'C': 1, 'WHERE': 2, 'AS': 2, 'TEMP': 1, 'GROUP': 1, 'BY': 1}
{'SELECT': 3, 'FROM': 3, 'WHERE': 3, 'IN': 1, 'AS': 1, 'TEMP': 1, 'MAX': 1}
{'SELECT': 1, 'FROM': 1, 'NATURAL': 1, 'JOIN': 1, 'C': 1, 'WHERE': 1}
